In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from sklearn.linear_model import Lasso
# fit a logistic regression model on an imbalanced classification dataset
from sklearn.metrics import r2_score, roc_auc_score, recall_score, precision_score, accuracy_score, mean_squared_error, auc, roc_curve
from numpy import mean
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.model_selection import GridSearchCV
from sklearn.utils.class_weight import compute_class_weight
import scipy.stats as st
from joblib import dump, load
import pickle
import csv
import random
from matplotlib.transforms import Affine2D
from imblearn.over_sampling import SMOTE, ADASYN
import dimod
import json
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from copy import deepcopy

[Errno 17] File exists: 'todel'


In [20]:
# define directory structure
models_dir = "models"
models_evaluation_dir = "models_evaluation"
plots_dir = "plots"
extended_model_base_dir = os.path.join(models_dir, "extended_models")
extended_model_evaluation_base_dir = os.path.join(models_evaluation_dir, "extended_models")
condensed_model_base_dir = os.path.join(models_dir, "condensed_models")
condensed_model_evaluation_base_dir = os.path.join(models_evaluation_dir, "condensed_models")
p_matrix_base_dir = os.path.join(models_evaluation_dir, "p_matrix")
p_matrix_plot_base_dir = os.path.join(plots_dir,"p_matrix")

In [26]:
df_all = pd.read_csv("data/med_orginal2.csv",header=0, sep="\,").fillna(method = "ffill")


target_f = 'Graft loss 1 year'

mandatory_f = [
    'AKI - KDIGO 2012',
    'FSGS',
    'Reduction to steroid only',
    'Transfusion [YES/NO]',
]

with open("data/16.json") as file:
    miqubo_result = dimod.SampleSet.from_serializable(json.load(file))
    miqubo_f = [x for x, y in miqubo_result.first.sample.items() if y ==1]

miqubo_f = list(set(miqubo_f) - set(mandatory_f) - set(target_f))
assert sum(df_all.isna().sum())==0, "Still nan entries in dataset"

df_ = deepcopy(df_all)
naming_map_ = {x[1] : "f_{}".format(x[0]) for x in enumerate(df_.columns)}
df_ = df_.rename(columns=naming_map_)
target_f = naming_map_[target_f]
mandatory_f = [naming_map_[x] for x in mandatory_f]
free_f_miqubo = [naming_map_[x] for x in miqubo_f]
free_f_all_ = list(set(naming_map_.values()) - set(mandatory_f) - set(target_f))

# get raw dataset
df_all = df_all.rename(columns = naming_map_)
df_miqubo = df_all[[target_f] + mandatory_f + free_f_miqubo]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [32]:
def normalize_dataset(df): 
    binary_f = [x for x in df.columns if len(np.unique(df[x]))==2]
    float_f = list(set([x for x, y in df.dtypes.items() if y == np.float]) - set(binary_f))
    int_f = list(set([x for x, y in df.dtypes.items() if y == np.int]) - set(binary_f))

    df_float_norm = pd.DataFrame(data=preprocessing.StandardScaler().fit(df[float_f]).transform(df[float_f]), columns=float_f)
    df_int_norm = pd.DataFrame(data=preprocessing.StandardScaler().fit(df[int_f]).transform(df[int_f]), columns=int_f)
    return pd.concat([df[binary_f], df_float_norm, df_int_norm], axis=1, join="inner")

def sample_dataset(df, target_f):
    # get normalized dataset
    y_train = df.pop(target_f)
    x_train = df

    # get normalized and sampled dataset
    df_sampled, label_sampled = SMOTE().fit_resample(x_train, y_train)
    df_sampled.insert(0, target_f, label_sampled)
    return df_sampled

def transform_dataset(df, target_f = target_f, normalized=True, sampled=True):
    df_ = deepcopy(df)
    if normalized:
        df_ = normalize_dataset(df_)
    if sampled:
        df_ = sample_dataset(df_, target_f)

    y_train = df_.pop(target_f)
    x_train = df_
    return train_test_split(x_train, y_train, test_size=0.2, random_state=42, shuffle=True)
   

x_train_raw, x_test_raw, y_train_raw, y_test_raw = transform_dataset(df_all)
x_train_raw, x_test_raw, y_train_raw, y_test_raw = transform_dataset(df_all, target_f, normalized = False, sampled=True)
x_train_raw, x_test_raw, y_train_raw, y_test_raw = transform_dataset(df_all, target_f, normalized = True, sampled=False)
x_train_raw, x_test_raw, y_train_raw, y_test_raw = transform_dataset(df_all, target_f, normalized = True, sampled=True)


In [50]:
def train_models(df, models, models_names, target_f, miqubo = True, normalized = True, sampled=True):
    miqubo_label = "miqubo" if miqubo else "all"
    model_save_dir = "raw"
    if normalized:
        df = normalize_dataset(df)
    if sampled:
        df = sample_dataset(df, target_f)

    if sampled:
        if normalized: 
            model_save_dir = "sampled_normalized"
        else:
            model_save_dir = "sampled"
    else:
        if normalized:
            model_save_dir = "normalized"
    try: 
        os.mkdir(os.path.join(models_dir, model_save_dir)) 
    except OSError as error: 
        print(error) 

    try: 
        os.mkdir(os.path.join(models_evaluation_dir, model_save_dir)) 
    except OSError as error: 
        print(error) 

    y_train = df.pop(target_f)
    x_train = df
    x_train, x_test, y_train, y_test =train_test_split(x_train, y_train, test_size=0.2, random_state=42, shuffle=True)
    trained_models = {}
    threashold = 0.9

    # define evaluation procedure
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

    for model, model_name in zip(models, models_names):
        # evaluate model
        scores = cross_val_score(model, x_train, y_train, scoring='roc_auc', cv=cv, n_jobs=-1)
        
        model.fit(x_train, y_train)
        model_path = os.path.join(models_dir, model_save_dir, '{}_{}.joblib'.format(model_name, miqubo_label))
        pickle.dump(
            model, 
            open(model_path, 'wb')
            )
        # discretize predictions
        y_pred = np.where(model.predict(x_test)>threashold,1, 0)

        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        roc = roc_auc_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred) 
        precission = precision_score(y_test, y_pred) 
        accuracy = accuracy_score(y_test, y_pred)

        fpr, tpr, thresholds = roc_curve(y_test, model.predict(x_test))

        # add model to dictionary
        trained_models[model_name] = {}
        trained_models[model_name]["model"] = model
        trained_models[model_name]["name"] = model_name
        trained_models[model_name]["pred"] = model.predict(x_test)
        trained_models[model_name]["predT"] =y_pred
        trained_models[model_name]["mse"] = mse
        trained_models[model_name]["r2"] = r2
        trained_models[model_name]["roc"] = roc
        trained_models[model_name]["recall"] = recall
        trained_models[model_name]["precission"] = precission
        trained_models[model_name]["accurarcy"] = accuracy
        trained_models[model_name]["fpr"] = fpr
        trained_models[model_name]["tpr"] = tpr 
        trained_models[model_name]["thresholds"] = thresholds

        # summarize performance
        #print('Mean ROC AUC: %.3f' % mean(scores))
        #print("Name: {} | ROC: {}, recall: {}, precission: {}, accuracy: {}".format(name, roc, recall, precission, accuracy))

    with open(os.path.join(
        models_evaluation_dir, model_save_dir,
        '{}.csv'.format(miqubo_label)), 
        'w', encoding='UTF8') as f:
        # create the csv writer
        writer = csv.writer(f)

        # write the header
        writer.writerow(['name', 'mse', 'r2', 'roc', 'recall', 'precission', 'accurarcy'])

        for model in trained_models:
            model_evaluation = trained_models[model_name]
            row = [
                model_evaluation['name'],
                model_evaluation['mse'],
                model_evaluation['r2'],
                model_evaluation['roc'],
                model_evaluation['recall'],
                model_evaluation['precission'],
                model_evaluation['accurarcy'],
            ]
            # write the data
            writer.writerow(row)

    return trained_models

In [51]:
models_names = ["Lasso", "Enet", "Loger", "Nearest Neighbors", "Linear SVM", "RBF SVM", "Gaussian Process",
        "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
        "Naive Bayes", "QDA"]
alpha = 0.01
models = [
    linear_model.Lasso(alpha=alpha, normalize = False),
    linear_model.ElasticNet(alpha=alpha, l1_ratio=0.7, normalize = False),
    LogisticRegression(solver='lbfgs', class_weight={x:y for x, y in zip(np.unique(df_all[target_f]), np.bincount(df_all[target_f]))}),
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1, max_iter=1000),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]
    
miqubo_trained_models_sampled            = train_models(deepcopy(df_all), models, models_names, target_f,miqubo = True,  normalized = False, sampled = True)
miqubo_trained_models                    = train_models(deepcopy(df_all), models, models_names, target_f,miqubo = True,  normalized = False, sampled = False)
miqubo_trained_models_normalized         = train_models(deepcopy(df_all), models, models_names, target_f,miqubo = True,  normalized = True,  sampled = False)
miqubo_trained_models_sampled_normalized = train_models(deepcopy(df_all), models, models_names, target_f,miqubo = True,  normalized = True,  sampled = True)
all_trained_models                       = train_models(deepcopy(df_all), models, models_names, target_f,miqubo = False, normalized = False, sampled = False)
all_trained_models_sampled               = train_models(deepcopy(df_all), models, models_names, target_f,miqubo = False, normalized = False, sampled = True)
all_trained_models_normalized            = train_models(deepcopy(df_all), models, models_names, target_f,miqubo = False, normalized = True,  sampled = False)
all_trained_models_sampled_normalized    = train_models(deepcopy(df_all), models, models_names, target_f,miqubo = False, normalized = True,  sampled = True)

[Errno 17] File exists: 'models/sampled'
[Errno 17] File exists: 'models_evaluation/sampled'


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/gaussian_process/kernels.py:418: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to

[Errno 17] File exists: 'models/raw'
[Errno 17] File exists: 'models_evaluation/raw'


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use 

[Errno 17] File exists: 'models/sampled'
[Errno 17] File exists: 'models_evaluation/sampled'


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/gaussian_process/kernels.py:418: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


[Errno 17] File exists: 'models/normalized'
[Errno 17] File exists: 'models_evaluation/normalized'


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn(

[Errno 17] File exists: 'models/sampled_normalized'
[Errno 17] File exists: 'models_evaluation/sampled_normalized'


/usr/local/lib/python3.6/dist-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


In [52]:
def auc(X, Y):
    return 1/(len(X)*len(Y)) * sum([kernel(x, y) for x in X for y in Y])

def kernel(X, Y):
    return .5 if Y==X else int(Y < X)

def structural_components(X, Y):
    V10 = [1/len(Y) * sum([kernel(x, y) for y in Y]) for x in X]
    V01 = [1/len(X) * sum([kernel(x, y) for x in X]) for y in Y]
    return V10, V01
    
def get_S_entry(V_A, V_B, auc_A, auc_B):
    return 1/(len(V_A)-1) * sum([(a-auc_A)*(b-auc_B) for a,b in zip(V_A, V_B)])

def z_score(var_A, var_B, covar_AB, auc_A, auc_B):
    return (auc_A - auc_B)/((var_A + var_B - 2*covar_AB)**(.5))


# Model A (random) vs. "good" model B
#preds_A = np.array([.5, .5, .5, .5, .5, .5, .5, .5, .5, .5])
#preds_B = np.array([.2, .5, .1, .4, .9, .8, .7, .5, .9, .8])
#actual= np.array([0, 0, 0, 0, 1, 0, 1, 1, 1, 1])

def group_preds_by_label(preds, actual):
    X = [p for (p, a) in zip(preds, actual) if a]
    Y = [p for (p, a) in zip(preds, actual) if not a]
    return X, Y

def get_p_value(preds_A, preds_B, actual_A, actual_B):
    X_A, Y_A = group_preds_by_label(preds_A, actual_A)
    X_B, Y_B = group_preds_by_label(preds_B, actual_B)
    V_A10, V_A01 = structural_components(X_A, Y_A)
    V_B10, V_B01 = structural_components(X_B, Y_B)
    auc_A = auc(X_A, Y_A)
    auc_B = auc(X_B, Y_B)
    # Compute entries of covariance matrix S (covar_AB = covar_BA)
    var_A = (get_S_entry(V_A10, V_A10, auc_A, auc_A) * 1/len(V_A10)
            + get_S_entry(V_A01, V_A01, auc_A, auc_A) * 1/len(V_A01))
    var_B = (get_S_entry(V_B10, V_B10, auc_B, auc_B) * 1/len(V_B10)
            + get_S_entry(V_B01, V_B01, auc_B, auc_B) * 1/len(V_B01))
    covar_AB = (get_S_entry(V_A10, V_B10, auc_A, auc_B) * 1/len(V_A10)
                + get_S_entry(V_A01, V_B01, auc_A, auc_B) * 1/len(V_A01))
    # Two tailed test
    z = z_score(var_A, var_B, covar_AB, auc_A, auc_B)
    p = st.norm.sf(abs(z))*2
    return p


In [53]:
def get_p_matrix(df, target_f, trained_models_a, trained_models_b, miqubo_a = True, miqubo_b = True, normalized_a = True, normalized_b = True, sampled_a=True, sampled_b=True):
    df_a = deepcopy(df)
    df_b = deepcopy(df)
    if normalized_a:
        df_a = normalize_dataset(df_a)
    if sampled_a:
        df_a = sample_dataset(df_a, target_f)
    if normalized_b:
        df_b = normalize_dataset(df_b)
    if sampled_a:
        df_b = sample_dataset(df_b, target_f)

    y_train_a = df_a.pop(target_f)
    x_train_a = df_a
    x_train, x_test_a, y_train, y_test_a = train_test_split(x_train_a, y_train_a, test_size=0.2, random_state=42, shuffle=True)

    label_a = "Miqubo" if miqubo_a else "All"
    label_a += " features over "
    
    if sampled_a:
        if normalized_a: 
            label_a = "sampled and normalized"
        else:
            label_a = "sampled"
    else:
        if normalized_a:
            label_a = "normalized"
    label_a += " dataset"

    label_b = "Miqubo" if miqubo_a else "All"
    label_b += " features over "
    
    if sampled_b:
        if normalized_b: 
            label_b = "sampled and normalized"
        else:
            label_b = "sampled"
    else:
        if normalized_b:
            label_b = "normalized"
    label_b += " dataset"

    
    n_models = len(trained_models_a)
    p_matrix = np.zeros((n_models, n_models))
    for row, model_A in enumerate(trained_models_a):
        preds_A = trained_models_a[model_A]['model'].predict(x_test_a)
        for col, model_B in enumerate(trained_models_b):
            preds_B = trained_models_b[model_B]['model'].predict(x_test_b)
            p_matrix[row][col] = get_p_value(preds_A, preds_B, y_test_a, y_test_b)
    
    # save p_matrix to csv
    np.savetxt(os.path.join(p_matrix_base_dir, file_name+".csv"), p_matrix, delimiter = ',')

    # save

    return p_matrix